<a href="https://colab.research.google.com/github/csrajath/CSC_8980_NLP/blob/main/rcs1_TNLP_CSC_8980_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rajath Chikkatur Srinivasa
### 002552425

In [2]:
# importing libraries
import nltk, os
import pandas as pd
from collections import defaultdict
import random
from nltk.corpus import stopwords
from IPython.display import clear_output
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# models
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
#sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
# unzipping dataset
!tar -xvf /content/drive/MyDrive/Projects/NLP/review_polarity.tar.gz
clear_output()

1. Using NLTK tokenize all documents, separated by polarity, remove stop words, and list the top 20 most frequent tokens (and their counts) for the positive reviews, and the top 20 most frequent tokens (and their counts). What kind of things do you notice are different between the two sets? (30 points)

In [4]:
# collecting stopwords and punkts
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [5]:
pos_filepath = '/content/txt_sentoken/pos/'
neg_filepath = '/content/txt_sentoken/neg/'
pos_content = []
neg_content = []

In [6]:
for i in os.listdir(pos_filepath):
    file_content = open(os.path.join(pos_filepath,i), 'r').read()
    pos_content.append(file_content)
complete_pos_content = " ".join(pos_content)

In [7]:
for i in os.listdir(neg_filepath):
    file_content = open(os.path.join(neg_filepath,i), 'r').read()
    neg_content.append(file_content)
complete_neg_content = " ".join(neg_content)

In [8]:
#tokenizing positive
pos_tokens = word_tokenize(complete_pos_content) 
#tokenizing negative
neg_tokens = word_tokenize(complete_neg_content) 

In [9]:
#remove stopwords from positive tokens
rsw_pos_tokens = [word for word in pos_tokens if word.lower() not in stop_words]
#remove stopwords from negative tokens
rsw_neg_tokens = [word for word in neg_tokens if word.lower() not in stop_words]

In [10]:
p= defaultdict(int)
for i in rsw_pos_tokens:
    p[i]+=1
n= defaultdict(int)
for i in rsw_neg_tokens:
    n[i]+=1

In [11]:
pos_sort = sorted(p.items(), key=lambda x: x[1], reverse=True)[:20]
neg_sort = sorted(n.items(), key=lambda x: x[1], reverse=True)[:20]

In [12]:
print("top 20 most frequent tokens and their count for positive reviews are:")
for i in pos_sort:
  print('token:', i[0], 'count:',i[1])

top 20 most frequent tokens and their count for positive reviews are:
token: , count: 42448
token: . count: 33714
token: 's count: 9473
token: `` count: 8494
token: ) count: 6039
token: ( count: 6014
token: film count: 5186
token: one count: 2943
token: n't count: 2775
token: movie count: 2497
token: like count: 1713
token: ? count: 1570
token: : count: 1502
token: story count: 1231
token: also count: 1200
token: good count: 1190
token: even count: 1175
token: time count: 1171
token: would count: 1079
token: character count: 1067


In [13]:
print("top 20 most frequent tokens and their count for negative reviews are:")
for j in neg_sort:
  print('token:', j[0], 'count:',j[1])

top 20 most frequent tokens and their count for negative reviews are:
token: , count: 35269
token: . count: 32162
token: `` count: 9123
token: 's count: 8655
token: ) count: 5742
token: ( count: 5650
token: film count: 4257
token: n't count: 3442
token: movie count: 3174
token: one count: 2637
token: ? count: 2201
token: like count: 1832
token: : count: 1540
token: even count: 1381
token: would count: 1185
token: good count: 1126
token: time count: 1111
token: ! count: 1056
token: get count: 1039
token: bad count: 1019


* What kind of things do you notice are different between the two sets?
- Firstly the sentiment terms count and the term itself is different. This is obvious. For example: good v/s bad
- There are some positive tokens that are appearing the negative sentiment list like good.
- The ordering of the terms are different. 
- There are some terms present in positive list but not in negative list and vice-versa

2. Using the code from previous lectures, build 3 polarity classifiers using the following parameters (20 points). Note: just train the models.\
a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.\
b) For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.\
c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer

a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [14]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m1 = len(pos_content[:int((50/100)*len(pos_content))])
neg_m1 = len(neg_content[:int((70/100)*len(neg_content))])
m1_pos_train = pos_content[:pos_m1]
m1_neg_train = neg_content[:neg_m1]
m1_pos_test = pos_content[pos_m1:]
m1_neg_test = neg_content[neg_m1:]

In [15]:
m1_train_data = m1_pos_train + m1_neg_train
m1_train_labels_1 = [1 for i in m1_pos_train]
m1_train_labels_0 = [0 for i in m1_neg_train]
m1_test_labels_1 = [1 for i in m1_pos_test]
m1_test_labels_0 = [0 for i in m1_neg_test]
m1_train_labels = m1_train_labels_1 + m1_train_labels_0 # 500-pos; 700-neg
m1_test_labels = m1_test_labels_1 + m1_test_labels_0 # 500-pos; 300-neg
#training NaiveBayes1
model_nb_1 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_1.fit(m1_train_data, m1_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

b) For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [16]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m2 = len(pos_content[:int((70/100)*len(pos_content))])
neg_m2 = len(neg_content[:int((50/100)*len(neg_content))])
m2_pos_train = pos_content[:pos_m2]
m2_neg_train = neg_content[:neg_m2]
m2_pos_test = pos_content[pos_m2:]
m2_neg_test = neg_content[neg_m2:]

In [17]:
#training NaiveBayes2
m2_train_data = m2_pos_train + m2_neg_train
m2_train_labels_1 = [1 for i in m2_pos_train]
m2_train_labels_0 = [0 for i in m2_neg_train]
m2_test_labels_1 = [1 for i in m2_pos_test]
m2_test_labels_0 = [0 for i in m2_neg_test]
m2_train_labels = m2_train_labels_1 + m2_train_labels_0
# m2_test_labels = m2_test_labels_1 + m2_test_labels_0
model_nb_2 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_2.fit(m2_train_data, m2_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer

In [18]:
random.shuffle(pos_content)
random.shuffle(neg_content)
pos_m3 = len(pos_content[:int((25/100)*len(pos_content))])
neg_m3 = len(neg_content[:int((25/100)*len(neg_content))])
m3_pos_train = pos_content[:pos_m3]
m3_neg_train = neg_content[:neg_m3]
m3_pos_test = pos_content[pos_m3:]
m3_neg_test = neg_content[neg_m3:]

In [19]:
#training SVM
m3_train_data = m3_pos_train + m3_neg_train
m3_train_labels_1 = [1 for i in m3_pos_train]
m3_train_labels_0 = [0 for i in m3_neg_train]
m3_test_labels_1 = [1 for i in m3_pos_test]
m3_test_labels_0 = [0 for i in m3_neg_test]
m3_train_labels = m3_train_labels_1 + m3_train_labels_0
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
model_svc_3 = make_pipeline(TfidfVectorizer(), SVC())
model_svc_3.fit(m3_train_data, m3_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

3) Using the models from question 2, evaluate them on their individual rest of the dataset. This is, \
for a) 50% positive and 30% negative, \
for b) 50% negative and 30% positive, and \
for c) 75% negative and 75% positive. Calculate and show ONLY the following metrics for each model: Accuracy, Precision, Recall, Macro F1-score. (15 points)

In [20]:
# printing performance metrics for model-1
m1_name = 'NaiveBayes1-TfIdf'
m1_pred = model_nb_1.predict(m1_pos_test+m1_neg_test)
m1_accuracy = accuracy_score(m1_test_labels, m1_pred)
m1_precision = precision_score(m1_test_labels, m1_pred)
recall_model1 =  recall_score(m1_test_labels, m1_pred)
mf1_model_1 =  f1_score(m1_pred, m1_test_labels, average='macro')
m1_pvalues = [m1_name, m1_accuracy, m1_precision, recall_model1, mf1_model_1]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [21]:
# printing performance metrics for model-2
m2_test_labels = m2_test_labels_1 + m2_test_labels_0
m2_name = 'NaiveBayes2-TfIdf'
m2_pred = model_nb_2.predict(m2_pos_test+m2_neg_test)
m2_accuracy = accuracy_score(m2_test_labels,m2_pred)
m2_precision = precision_score(m2_test_labels, m2_pred,m2_test_labels)
recall_model2 =  recall_score(m2_test_labels,m2_pred)
mf1_model_2 =  f1_score(m2_pred, m2_test_labels, average='macro')
m2_pvalues = [m2_name, m2_accuracy, m2_precision, recall_model2, mf1_model_2]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [22]:
# printing performance metrics for model-3
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
m3_name = 'Support Vector Machine'
m3_pred = model_svc_3.predict(m3_pos_test+m3_neg_test)
m3_accuracy = accuracy_score(m3_test_labels, m3_pred)
m3_precision = precision_score(m3_test_labels, m3_pred)
recall_model3 =  recall_score(m3_test_labels, m3_pred)
mf1_model_3 =  f1_score(m3_pred,m3_test_labels, average='macro')
m3_pvalues = [m3_name, m3_accuracy, m3_precision, recall_model3, mf1_model_3]
# print('Mean Absolute Error:', mean_absolute_error(test.target,labels))
# print('Mean Squared Error:', mean_squared_error(test.target, labels))  
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(test.target, labels)))

In [23]:
#creating a dataframe for metrics of all models performance metrixs
pm_list = [m3_pvalues, m2_pvalues, m1_pvalues]
pm_df = pd.DataFrame(pm_list, columns = ["model_name", "accuracy", "precison", "recall", 'macro_f1_score'])
pm_df

,model_name,accuracy,precison,recall,macro_f1_score
0,Support Vector Machine,0.78400,0.768262,0.813333,0.783814
1,NaiveBayes2-TfIdf,0.37625,0.375469,1.000000,0.274971
2,NaiveBayes1-TfIdf,0.37625,1.000000,0.002000,0.274971


4) Using the model performance metrics from question 3, answer the following questions. Please provide logical and intuitive rationale for your answers, simple answers like: because it has the best score, will not be sufficient. (40 points):

a) What is the best performing model?\
Support vector machines is the best performing model.

b) Why do you think this is the best performing model?\
The training set for the category of sentiments (positve and negative) in SVM is equally distributed that is well balanced

c) How does class imbalance play in determining polarity?\
Since model1 and model2 had class imbalances. It can be seen from the derived metrics for them that precision and recall respectively for each of them is calculated as 1 which is not an ideal scinario for a classification task. This is also reflected in their accuracy scores which is low. Over all, with respect to polarity - Model 1 and Model 2 , due to class imbalance, classifies their own polarity(higher percentage polarity dataset) correctly.  In this case, Negative(Precision-1 in model1) and Positive (Precision-1 in model2) respectively

d) Do you think either more data or a better model is a better approach for this
kind of task?\
More data does not always guarentee better performance/output. It is a blend of having the correct model with right amount of data. Sometimes a better performing model provides incorrect output due to improper balance in dataset. An hybrid model approach can be incorporated. Transformer models, Neural network based models can be trained to achieve 

5) Using NLTK and VADER, calculate the sentiment score for all documents in the
positive polarity. Calculate the polarity threshold needed (and reasonable) to have the majority of the document labels match. Do the same for the negative class. Provide the threshold needed, the reason why you think this threshold is reasonable, and the accuracy percentage (how many documents are correctly labeled using this threshold). (45 points):

In [24]:
sia = SIA()

In [25]:
def get_scores(content, filename):
    sia_scores = sia.polarity_scores(content)
    
    return pd.Series({
        'filename': filename,
        'compound': sia_scores['compound'],
        'positive': sia_scores['pos'],
        'neutral': sia_scores['neu'],
        'negative': sia_scores['neg']
    })

In [26]:
df_pos_scores = pd.DataFrame([])

for i in os.listdir(pos_filepath):
    file_content = open(os.path.join(pos_filepath,i), 'r').read()
    df_1 = get_scores(file_content, i).to_frame().transpose()
    for index, row in df_1.iterrows():
      df_pos_scores = df_pos_scores.append({'file_name': row['filename'], 'compound': row['compound'],'positive': row['positive'], 'neutral': row['neutral'], 'negative': row['negative']}, ignore_index = True)

In [27]:
print('The sentiment score for all documents in the positive polarity is saved in the below dataframe')
df_pos_scores

The sentiment score for all documents in the positive polarity is saved in the below dataframe


,compound,file_name,negative,neutral,positive
0,0.9956,cv684_11798.txt,0.078,0.761,0.162
1,0.9955,cv990_11591.txt,0.064,0.827,0.109
2,0.9957,cv187_12829.txt,0.062,0.764,0.174
3,0.8974,cv441_13711.txt,0.119,0.749,0.132
4,0.9997,cv967_5788.txt,0.059,0.734,0.207
...,...,...,...,...,...
995,0.9997,cv521_15828.txt,0.080,0.671,0.249
996,0.9959,cv822_20049.txt,0.059,0.801,0.139
997,0.9965,cv710_22577.txt,0.045,0.817,0.138
998,-0.9331,cv771_28665.txt,0.117,0.771,0.112


In [28]:
df_neg_scores = pd.DataFrame([])

for i in os.listdir(neg_filepath):
    file_content = open(os.path.join(neg_filepath,i), 'r').read()
    df_2 = get_scores(file_content, i).to_frame().transpose()
    for index, row in df_2.iterrows():
      df_neg_scores = df_neg_scores.append({'file_name': row['filename'], 'compound': row['compound'],'positive': row['positive'], 'neutral': row['neutral'], 'negative': row['negative']}, ignore_index = True)

In [29]:
print('The sentiment score for all documents in the negative polarity is saved in the below dataframe')
df_neg_scores

The sentiment score for all documents in the negative polarity is saved in the below dataframe


,compound,file_name,negative,neutral,positive
0,-0.5308,cv496_11185.txt,0.102,0.806,0.092
1,-0.9941,cv265_11625.txt,0.153,0.730,0.116
2,-0.7614,cv436_20564.txt,0.096,0.820,0.084
3,-0.9300,cv708_28539.txt,0.119,0.794,0.087
4,-0.9926,cv130_18521.txt,0.184,0.726,0.089
...,...,...,...,...,...
995,-0.9781,cv570_28960.txt,0.113,0.822,0.065
996,0.9446,cv110_27832.txt,0.115,0.717,0.168
997,0.8970,cv983_24219.txt,0.105,0.760,0.135
998,0.9780,cv493_14135.txt,0.051,0.838,0.111


Provide the threshold needed, the reason why you think this threshold is reasonable, and the accuracy percentage (how many documents are correctly labeled using this threshold). (45 points):

In [ ]:
# one way of calculating the threshold is using the mean of values for each polarity and derive standard deviation to deduce accuracy
df_pos_mean = df_pos_scores['positive'].mean()
df_pos_mean = df_pos_scores['negative'].mean()
df_neg_mean = df_neg_scores['negative'].mean()

Bonus (40 points): Repeat questions 2,3 and 4 removing all stopwords. Answer the
following questions: Did this change the results in any way? Why do you think so?

In [30]:
pos_content_bonus = []
# removing stop words for positive review
# rsw_pos_tokens = [word for word in pos_tokens if word.lower() not in stop_words]
for i in pos_content:
  sw_tokens_pos = []
  doc_tokens = nltk.word_tokenize(i)
  for token in doc_tokens:
    if token not in stop_words:
      sw_tokens_pos.append(token)
  pos_content_bonus.append(' '.join(sw_tokens_pos))

In [31]:
neg_content_bonus = []
# removing stop words for positive review
# rsw_pos_tokens = [word for word in pos_tokens if word.lower() not in stop_words]
for i in neg_content:
  sw_tokens_neg = []
  doc_tokens = nltk.word_tokenize(i)
  for token in doc_tokens:
    if token not in stop_words:
      sw_tokens_neg.append(token)
  neg_content_bonus.append(' '.join(sw_tokens_neg))

a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [45]:
random.shuffle(pos_content_bonus)
random.shuffle(neg_content_bonus)

pos_m3 = len(pos_content_bonus[:int((50/100)*len(pos_content_bonus))])
neg_m3 = len(neg_content_bonus[:int((70/100)*len(neg_content_bonus))])
m1_pos_train = pos_content_bonus[:pos_m3]
m1_neg_train = neg_content_bonus[:neg_m3]
m1_pos_test = pos_content_bonus[pos_m3:]
m1_neg_test = neg_content_bonus[neg_m3:]

700

In [46]:
m1_train_data = m1_pos_train + m1_neg_train
m1_train_labels_1 = [1 for i in m1_pos_train]
m1_train_labels_0 = [0 for i in m1_neg_train]
m1_test_labels_1 = [1 for i in m1_pos_test]
m1_test_labels_0 = [0 for i in m1_neg_test]
m1_train_labels = m1_train_labels_1 + m1_train_labels_0 # 500-pos; 700-neg
m1_test_labels = m1_test_labels_1 + m1_test_labels_0 # 500-pos; 300-neg
#training NaiveBayes1 for bonus
model_nb_1 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_1.fit(m1_train_data, m1_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

Evaluate above models on their individual rest of the dataset. This is,
for 50% positive and 30% negative,

In [200]:
# printing performance metrics for model-1
m1_name = 'NaiveBayes1-TfIdf'
m1_pred = model_nb_1.predict(m1_pos_test+m1_neg_test)
m1_accuracy = accuracy_score(m1_test_labels, m1_pred)
m1_precision = precision_score(m1_test_labels, m1_pred)
recall_model1 =  recall_score(m1_test_labels, m1_pred)
mf1_model_1 =  f1_score(m1_pred, m1_test_labels, average='macro')
m1_pvalues = [m1_name, m1_accuracy, m1_precision, recall_model1, mf1_model_1]

b) For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

In [47]:
random.shuffle(pos_content_bonus)
random.shuffle(neg_content_bonus)

pos_m3 = len(pos_content_bonus[:int((70/100)*len(pos_content_bonus))])
neg_m3 = len(neg_content_bonus[:int((50/100)*len(neg_content_bonus))])
m2_pos_train = pos_content_bonus[:pos_m3]
m2_neg_train = neg_content_bonus[:neg_m3]
m2_pos_test = pos_content_bonus[pos_m3:]
m2_neg_test = neg_content_bonus[neg_m3:]

In [48]:
#training NaiveBayes2 for bonus
m2_train_data = m2_pos_train + m2_neg_train
m2_train_labels_1 = [1 for i in m2_pos_train]
m2_train_labels_0 = [0 for i in m2_neg_train]
m2_test_labels_1 = [1 for i in m2_pos_test]
m2_test_labels_0 = [0 for i in m2_neg_test]
m2_train_labels = m2_train_labels_1 + m2_train_labels_0
# m2_test_labels = m2_test_labels_1 + m2_test_labels_0
model_nb_2 = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb_2.fit(m2_train_data, m2_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

Evaluate above models on their individual rest of the dataset. This is,
for b) 50% negative and 30% positive,

In [201]:
# printing performance metrics for model-2
m2_test_labels = m2_test_labels_1 + m2_test_labels_0
m2_name = 'NaiveBayes2-TfIdf'
m2_pred = model_nb_2.predict(m2_pos_test+m2_neg_test)
m2_accuracy = accuracy_score(m2_test_labels,m2_pred)
m2_precision = precision_score(m2_test_labels, m2_pred,m2_test_labels)
recall_model2 =  recall_score(m2_test_labels,m2_pred)
mf1_model_2 =  f1_score(m2_pred, m2_test_labels, average='macro')
m2_pvalues = [m2_name, m2_accuracy, m2_precision, recall_model2, mf1_model_2]

c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer

In [49]:
random.shuffle(pos_content_bonus)
random.shuffle(neg_content_bonus)

pos_m3 = len(pos_content_bonus[:int((25/100)*len(pos_content_bonus))])
neg_m3 = len(neg_content_bonus[:int((25/100)*len(neg_content_bonus))])
m3_pos_train = pos_content_bonus[:pos_m3]
m3_neg_train = neg_content_bonus[:neg_m3]
m3_pos_test = pos_content_bonus[pos_m3:]
m3_neg_test = neg_content_bonus[neg_m3:]

In [50]:
#training SVM fo rbonus
m3_train_data = m3_pos_train + m3_neg_train
m3_train_labels_1 = [1 for i in m3_pos_train]
m3_train_labels_0 = [0 for i in m3_neg_train]
m3_test_labels_1 = [1 for i in m3_pos_test]
m3_test_labels_0 = [0 for i in m3_neg_test]
m3_train_labels = m3_train_labels_1 + m3_train_labels_0
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
model_svc_3 = make_pipeline(TfidfVectorizer(), SVC())
model_svc_3.fit(m3_train_data, m3_train_labels)
# labels = model.predict(test.data)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

Evaluate above models on their individual rest of the dataset. This is, for b) 50% negative and 30% positive,

In [51]:
# printing performance metrics for model-3
m3_test_labels = m3_test_labels_1 + m3_test_labels_0
m3_name = 'Support Vector Machine'
m3_pred = model_svc_3.predict(m3_pos_test+m3_neg_test)
m3_accuracy = accuracy_score(m3_test_labels, m3_pred)
m3_precision = precision_score(m3_test_labels, m3_pred)
recall_model3 =  recall_score(m3_test_labels, m3_pred)
mf1_model_3 =  f1_score(m3_pred,m3_test_labels, average='macro')
m3_pvalues = [m3_name, m3_accuracy, m3_precision, recall_model3, mf1_model_3]

In [53]:
#creating a dataframe for metrics of all models performance metrixs
pm_list_bonus = [m3_pvalues, m2_pvalues, m1_pvalues]
pm_df_bonus = pd.DataFrame(pm_list_bonus, columns = ["model_name", "accuracy", "precison", "recall", 'macro_f1_score'])
pm_df_bonus

,model_name,accuracy,precison,recall,macro_f1_score
0,Support Vector Machine,0.791333,0.793289,0.788,0.791331
1,NaiveBayes2-TfIdf,0.376250,0.375469,1.000,0.274971
2,NaiveBayes1-TfIdf,0.376250,1.000000,0.002,0.274971


a) What is the best performing model?
Support vector machines is the best performing model.

b) Why do you think this is the best performing model?\
Because their classes are well balances

c) How does class imbalance play in determining polarity?\
Class imbalance leads to low accuracy which is seen in model 1(naivebayes1) and model 2 (naivebayes2)

d) Do you think either more data or a better model is a better approach for this kind of task?
As mentioned in Q4, increasing the data does not guarentee better results. In fact it might overfit/underfit. With respect to models, one can try advanced models like Neural Networks based models

e) Did this change the results in any way? Why do you think so?\
No so, because stop words does not majorly determine the polarity of a term. That is usually not how the sentiment libraries determine polarity.  The same stop words would be present in either/both dataset (positive and negative)